In [23]:
import mne
import pandas as pd
import numpy as np
import os
from mne.preprocessing import ICA

from scipy.signal import welch
from scipy.stats import kurtosis, skew
import pywt
import antropy as ant

Firstly, a single EEG is imported to explore its data.

In [2]:
raw = mne.io.read_raw_edf('data/MODMA_EEG_BIDS_format/EEG_LZU_2015_2_resting state/sub-001/eeg/sub-001_task-Resting-state_eeg.EDF')
raw.describe()
print(raw.ch_names)

Convert the electrode positions Excel file to a CSV one.

In [ ]:
excel_data = pd.read_excel("data/MODMA_EEG_BIDS_format/EEG_LZU_2015_2_resting state/subjects_information_EEG_128channels_resting_lanzhou_2015.xlsx")
excel_data.to_csv("data/MODMA_EEG_BIDS_format/EEG_LZU_2015_2_resting state/subject_information.csv")

Here the feature extraction functions for the raw EEG data are defined.

In [4]:
def band_powers_ratios(raw_frame, explicit=True): # raw frame is 128 x 75250
        
    freqs, psd = welch(raw_frame, fs=250, axis=1)

    freq_bands = {
        "Delta": [1, 4], 
        "Theta": [4, 8],
        "Alpha": [8, 13],
        "Beta":  [13, 30],
        "Gamma": [30, 45]
        }

    # Calculate global average band power for entire sample
    powers = {}
    for band, (l, h) in freq_bands.items():
        mask = (freqs >= l) & (freqs <= h)
        # Take average across both frequencies and channel dimensions to generate a single global average band power value
        powers[band] = 10 * np.log10(np.mean(psd[:, mask]) + 1e-12)  # Avoid log(0) and convert to decibel (standard for PSD)
    
    # Calculate ratios between global average band powers
    ratios = {
        f"{b1}/{b2}": powers[b1] / powers[b2]
        for i, b1 in enumerate(powers)
        for j, b2 in enumerate(powers)
        if i != j
    }
    
    merged = powers | ratios
    merged_dataframe = pd.DataFrame([merged])
    
    if explicit:
        print(merged_dataframe.shape)
        print(merged_dataframe)
        
    return merged_dataframe


# Discrete Wavelet Transform
def db_wavelet_features(raw_frame, wavelet="db4", level=4, explicit=True):
    features = []
    # Perform Discrete Wavelet Transform using Daubechies
    coefs = pywt.wavedec(raw_frame, wavelet=wavelet, level=level)
    for i, c in reversed(list(enumerate(coefs))):
        features.extend([
            np.sqrt(np.mean(np.square(c))),
            kurtosis(c.flatten()),
            skew(c.flatten())
        ])
        
    # Extracting global wavelet entropy
    
    # Calcuate energy per wavelet
    energies = np.array([np.sum(np.square(c)) for c in coefs])
    # See how much they contribute to the overall signal
    distribution = energies / (np.sum(energies)+1e-12)
    # Calculate Shannon entropy on the distribution
    entropy = -np.sum(distribution * (np.log2(distribution)+1e-12))
    features.append(entropy) 
    
    # Generate feature labels
    labels = []
    labels.extend(["cA4_RMS", "cA4_kurtosis", "cA4_skew"])
    labels.extend([f"cD{i}_{metric}" for i in range(4,0,-1) for metric in ["RMS", "kurtosis", "skew"]])
    labels.append("wavelet_entropy")
    
    # Combine the data with the labels to create a dataframe
    final = pd.DataFrame(data=[features], columns=labels)

    if explicit:
        print(final.shape)
        print(final)
    
    return final

def other_metrics(raw_frame, explicit=True):  
    head_sections = {
        "frontal": [31, 24, 20, 13, 7, 0, 25, 21, 14, 8, 1, 26, 22, 17, 15, 9, 2, 23, 18, 10, 3, 123, 122, 124, 127, 16, 126, 125, 11, 4],
        "central": [19, 5, 117, 28, 12, 111, 110, 35, 29, 6, 105, 104, 103, 41, 36, 33, 79, 86, 92, 52, 53, 54, 78, 85],
        "temporal": [47, 42, 37, 32, 33, 27, 43, 38, 39, 34, 48, 44, 40, 45, 46, 55, 56, 49, 50, 51, 57, 62, 116, 115, 121, 120, 119, 118, 109, 114, 113, 108, 102, 107, 112, 97, 101, 91, 96, 100, 96, 99, 106, 98],
        "parietal": [63,58,59,60,64,65,66,69,70,61,71,74,75,82,76,83,77,84,89,90,94],
        "occipital": [67, 68, 72, 73, 80, 81, 87, 88, 93]
    }
    
    features = []
    labels = []
    passes = []
    
    # Global metrics (across all electrodes)
    # Spectral entropy - calculate for each electrode, then average
    spectral_entropies = []
    for i in range(raw_frame.shape[0]):
        try:
            spectral_entropies.append(ant.spectral_entropy(raw_frame.iloc[i, :], sf=250))
        except:
            passes.append(f"Spectral entropy pass at i={i}")
    features.append(np.mean(spectral_entropies) if spectral_entropies else np.nan)
    
    # Permutation entropy - calculate for each electrode, then average
    perm_entropies = []
    for i in range(raw_frame.shape[0]):
        try:
            perm_entropies.append(ant.perm_entropy(raw_frame.iloc[i, :]))
        except:
            passes.append(f"Permutation entropy pass at i={i}")
    features.append(np.mean(perm_entropies) if perm_entropies else np.nan)
    
    # Hjorth mobility and complexity - calculate for each electrode, then average
    mobility_values = []
    complexity_values = []
    for i in range(raw_frame.shape[0]):
        try:
            m, c = ant.hjorth_params(raw_frame.iloc[i, :])
            mobility_values.append(m)
            complexity_values.append(c)
        except:
            passes.append(f"Hjorth pass at i={i}")
    features.append(np.mean(mobility_values) if mobility_values else np.nan)
    features.append(np.mean(complexity_values) if complexity_values else np.nan)
    
    # Zero crossings - calculate for each electrode, then average
    zero_crossings = []
    for i in range(raw_frame.shape[0]):
        try:
            zero_crossings.append(ant.num_zerocross(raw_frame.iloc[i, :]))
        except:
            passes.append(f"Zero crossings pass at i={i}")
    features.append(np.mean(zero_crossings) if zero_crossings else np.nan)
    
    # Process each head section
    for section, indices in head_sections.items():
        # Process metrics for electrodes in this section
        sample_entropies = []
        higuchi_values = []
        dfa_values = []
        spectral_entropies_section = []
        perm_entropies_section = []
        mobility_section = []
        complexity_section = []
        zero_crossings_section = []
        
        for idx in indices:
            if idx < raw_frame.shape[0]:  # Make sure index is in range
                # Get the time series for this electrode
                electrode_data = raw_frame.iloc[idx, :].to_numpy(dtype=np.float64)
                electrode_data = np.ascontiguousarray(electrode_data)
                
                # Sample entropy
                sample_entropies.append(ant.sample_entropy(electrode_data))

                # Higuchi Fractal Dimension
                higuchi_values.append(ant.higuchi_fd(electrode_data))

                # Detrended fluctuation analysis
                dfa_values.append(ant.detrended_fluctuation(electrode_data))

                # Spectral entropy for this section
                spectral_entropies_section.append(ant.spectral_entropy(electrode_data, sf=250))

                # Permutation entropy for this section
                perm_entropies_section.append(ant.perm_entropy(electrode_data))

                # Hjorth parameters for this section
                m, c = ant.hjorth_params(electrode_data)
                mobility_section.append(m)
                complexity_section.append(c)

                # Zero crossings for this section
                zero_crossings_section.append(ant.num_zerocross(electrode_data))

        # Add average metrics for this section
        features.append(np.mean(sample_entropies))
        features.append(np.mean(higuchi_values))
        features.append(np.mean(dfa_values))
        features.append(np.mean(spectral_entropies_section))
        features.append(np.mean(perm_entropies_section))
        features.append(np.mean(mobility_section))
        features.append(np.mean(complexity_section))
        features.append(np.mean(zero_crossings_section))
    
    # Create feature labels
    labels.extend([
        "global_spectral_entropy", 
        "global_permutation_entropy",
        "global_hjorth_mobility", 
        "global_hjorth_complexity", 
        "global_zero_crossings"
    ])
    
    for section in head_sections.keys():
        labels.extend([
            f"{section}_sample_entropy",
            f"{section}_higuchi",
            f"{section}_DFA",
            f"{section}_spectral_entropy",
            f"{section}_permutation_entropy",
            f"{section}_hjorth_mobility",
            f"{section}_hjorth_complexity",
            f"{section}_zero_crossings"
        ])
    
    # Replace any NaN values with 0
    features = [0 if np.isnan(x) else x for x in features]
    
    final = pd.DataFrame([features], columns=labels)
    # Also save the list of passes
    with open("passes.txt", "w") as file:
        for item in passes:
            file.write(f"{item}\n")
    
    if explicit:
        print(final.shape)
        print(final)
    
    return final
    
def overall_features(raw_frame, explicit=False): # This function combines the prior three feature extaction functions and returns a complete feature set for each data sample
    df1 = band_powers_ratios(raw_frame, explicit=explicit)
    df2 = db_wavelet_features(raw_frame, explicit=explicit)
    df3 = other_metrics(raw_frame, explicit=explicit)

    merged = pd.concat([df1, df2, df3], axis=1)
    
    if explicit:
        print(merged.shape)
        print()
        print(list(merged.columns))
        
    return merged

Make a montage (mapping of electrode positions to scalp) to use later, so that ICA is more accurate.

In [9]:
channel_names = [f"E{i}" for i in range(1,129)]

# The electrode locations are the same for all subjects
pos_file = pd.read_csv("data/MODMA_EEG_BIDS_format/EEG_LZU_2015_2_resting state/sub-001/eeg/sub-001_task-Resting-state_electrodes.tsv", sep="\t")
names = pos_file["name"].str.strip("'").to_list()
ch_pos = pos_file[["x","y","z"]].to_numpy()

pos_dic = {name:pos for name, pos in zip(names, ch_pos)}
montage = mne.channels.make_dig_montage(ch_pos=pos_dic, coord_frame="head")

This is where everything comes together for the preprocessing. All subject directories are iterated through and each EEG file is band-pass and notch filtered, the montage is applied, a synthetic EOG channel is created to remove ocular artifact removal from the data, ICA is ran on that, and the indepndent components are then used to reconstruct the original space. In the end, eveything is re-referenced to the average.

In [28]:
def preprocessing():
    # Create a list of the paths to each subject folder
    sub_folders = []
    for filename in os.listdir("data/MODMA_EEG_BIDS_format/EEG_LZU_2015_2_resting state/"):
        if "sub-" in filename:
            sub_folders.append(os.path.join(os.getcwd(), "data/MODMA_EEG_BIDS_format/EEG_LZU_2015_2_resting state/", filename))

    problematic = [] # potentially missing files
    all_data = [] 
    
    # For each subject folder 
    for subject in (sorted(sub_folders))[:2]:
        subject_id = os.path.basename(subject)
        edf_path = os.path.join(subject, "eeg", f"{subject_id}_task-Resting-state_eeg.EDF")
        print(edf_path)
        # If there is a missing file, skip this session
        if not os.path.exists(edf_path):
            problematic.append(subject_id)
            print("PROBLEMATIC: ", problematic[-1])
            continue           

        raw = mne.io.read_raw_edf(edf_path, preload=True)
        raw.filter(1.0, 45.0, picks="eeg") # band-pass filter
        raw.notch_filter(50, picks="eeg")  # notch filter power line noise (50hz because data was acquired in China)

        raw.set_montage(montage) # set the custom montage
        raw = mne.set_bipolar_reference(raw, anode='E22', cathode='E9', ch_name='EOG', drop_refs=False)  # synthetic EOG channel in order to carry out automatic component removal in ICA
        ica = ICA(n_components=None, method='fastica', random_state=23, max_iter='auto') # Create ICA object
        ica.fit(raw) # Fit it to raw data
        eog_inds, _ = ica.find_bads_eog(raw, ch_name='EOG')  # Find bad components based on the synthetic EOG channel we created
        ica.exclude.extend(eog_inds) # Remove bad components
        raw_clean = ica.apply(raw.copy()) # Reconstruct original space
        raw_clean.drop_channels(["EOG"]) # remove the synthetic channel we added
        raw_clean.set_eeg_reference('average', projection=False) # Re-reference to average

        raw_frame = raw_clean.get_data(picks="eeg").T  # Shape: (samples, channels)
        pd_frame = pd.DataFrame(raw_frame)
        
        all_data.append(pd_frame)
                
    return all_data

# mne.set_log_level('WARNING')  # Or 'ERROR' to suppress even more output 

Compute features from preprocessed data and combine all new features with the ones available for each subject (gender, age, research group, psych test scores, etc...). 

In [29]:
complete_set = pd.concat([overall_features(pd.DataFrame(x), explicit=False) for x in preprocessing()], ignore_index=True)
complete_set.to_csv("features_processed_only.csv", header=True, index=False)

In [30]:
processed = pd.read_csv("features_processed_only.csv")
csv_sub_info = pd.read_csv("data/MODMA_EEG_BIDS_format/EEG_LZU_2015_2_resting state/subject_information.csv")
final = pd.concat([csv_sub_info, processed], axis=1)
final.to_csv("data/MODMA_EEG_BIDS_format/EEG_LZU_2015_2_resting state/full_features.csv")